<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/6_2_Bert_Finetuning_sentence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 6.2 - BERT Finetuning para clasificación de textos

En esta segunda parte de la sesión vamos a ver como se puede reentrenar un modelo preentrenado de BERT/RoBERTa para poder conseguir un clasificador de texto como en las prácticas anteriores con el datasetEspañol.csv.

**Es importante tener en cuenta que para hacer el finetuning de modelos Transformers como BERT o RoBERTa es muy recomendable tener una GPU para poder reducir de manera drástica el tiempo de procesamiento. Por lo tanto se recomienda cargar este notebook en COLAB**

Lo primero que haremos será instalar las librerías de Huggingface transformers y datasets.


In [2]:
# Install libraries
!pip3 install -U transformers datasets torch
!pip3 install accelerate

# Descargamos nuestro datasetEspañol.csv
!wget https://dis.um.es/~valencia/recursosTGINE/datasetEspañol.csv

--2023-11-15 16:01:56--  https://dis.um.es/~valencia/recursosTGINE/datasetEspa%C3%B1ol.csv
Resolving dis.um.es (dis.um.es)... 155.54.239.5
Connecting to dis.um.es (dis.um.es)|155.54.239.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1953117 (1.9M) [text/csv]
Saving to: ‘datasetEspañol.csv’

datasetEspañol.csv  100%[===================>]   1.86M  1.64MB/s    in 1.1s    

2023-11-15 16:01:58 (1.64 MB/s) - ‘datasetEspañol.csv’ saved [1953117/1953117]



In [3]:
import transformers

# Modelo BERT en español - BETO
#path_bert_model = 'dccuchile/bert-base-spanish-wwm-uncased'
# Modelo BERT multilingüe
#path_bert_model = 'bert-base-multilingual-cased'
# Modelo BERTIN basado en RoBERTa
#path_bert_model = 'bertin-project/bertin-roberta-base-spanish'
# Modelo MarIA basado en RoBERTa
path_bert_model = 'PlanTL-GOB-ES/roberta-base-bne'

# Modelo "destilados" de BERT en español
#path_bert_model = 'CenIA/distillbert-base-spanish-uncased'

# Modelo AlBERT en español
#path_bert_model = 'CenIA/albert-base-spanish'

## Apartado 1.1 Cargarmos un modelo de BERT/RoBERTa preentrenado para clasificación binaria

A continuación vamos a cargar un modelo BERT/RoBERTa para clasificación binaria utilizando las librería KERAS de TensorFlow.

El finetuning y uso de BERT/RoBERTa se puede hacer tanto con Pytorch como con KERAS.

A continuación se muestra un ejemplo con KERAS que resulta más gráfico.

In [4]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
import json

# Cargamos un modelo de BERT preentrenado para clasificación. El número de etiquetas es 2
NUM_LABELS = 2
# La clase TFAutoModelForSequenceClassification es de Tensorflow y la AutoModelForSequenceClassification es de Pytorch
bert_class_model = AutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS)
# Cargamos el Tokenizer
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = bert_class_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= bert_class_model.config.id2label[predicted_class_id]
  print(text,'=>', prediction)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


hay muchos más muertos por covid => LABEL_0
el número de afectados por covid aumenta => LABEL_0
vamos a salir de la pandemia => LABEL_0
ánimo a todos => LABEL_0


## Apartado 1.2 Realizamos un Finetuning del modelo preentrenado de BERT/RoBERTa en Pytorch

Se puede observar por los resultados anteriores que este modelo de BERT/RoBERTA es un modelo general y no está adaptado para hacer clasificación de texto en 'positivo' y 'negativo' y mucho menos para el dominio del estado de alarma.

Vamos a cargar el dataset de siempre para poder hacer un finetuning de este modelo para mejorar de manera muy significativa los resultados del mismo.

Para el entrenamiento de los modelos de DeepLearning es necesario tener 3 conjuntos de datos: Train, Eval y Test.

In [5]:
import pandas
df = pandas.read_csv("datasetEspañol.csv",encoding="UTF-8")

p_train = 0.80 # Porcentaje de train.
p_eval = 0.20 # Porcentaje de eval.
p_test = 0.20 # Porcentaje de test

from sklearn.model_selection import train_test_split

df = df.sample(frac=1) #mezclamos el dataset


# Ponemos en lower_case en los tweets
df.tweet = df.tweet.apply (lambda x: x.lower())


# Para poder entrenar es necesario codificar las etiquetas como números. Para eso codificaremos
# los negativos con 0 y los positivos con 1
df['_label'] = df['label'].apply(lambda x : 1 if x == 'positive' else 0)

df_train, df_test = train_test_split (df, test_size = p_test)
df_train, df_eval = train_test_split (df_train, test_size = p_eval)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

Ejemplos usados para entrenar:  3813
Ejemplos usados para evaluar:  954
Ejemplos usados para test:  1192


A continuación preparamos los conjuntos de entrenamiento, evaluación y test en **Pythorch** para poder hacer el entrenamiento del modelo.

In [6]:
import torch
from transformers import AutoModelForSequenceClassification

# Cargamos el modelo para clasificación en Pytorch
bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS)

# Los datasets se preparan dde manera distinta a Tensorflow
class TGINEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments

batch_train_size = 16
batch_val_size = 64

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results',
  logging_dir = './logs',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size
)


tokenized_train_dataset = tokenizer (df_train.tweet.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.tweet.tolist (), truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.tweet.tolist (), truncation=True, padding = True)


# Como antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test._label.tolist())

from transformers import Trainer

trainer = Trainer (
    model = bert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()


print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Salvamos el modelo reentrenado
modelo ='modeloReentrenadoPytorch'
bert_class_model_pytorch.save_pretrained (modelo)
tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-2e0f96e1477f>:24: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Step,Training Loss
500,0.245400


PREDICCIONES SOBRE EVAL


{
  "eval_loss": 0.5743566751480103,
  "eval_accuracy": 0.8773584905660378,
  "eval_runtime": 6.2095,
  "eval_samples_per_second": 153.635,
  "eval_steps_per_second": 2.416,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.5519322156906128,
  "test_accuracy": 0.8791946308724832,
  "test_runtime": 9.521,
  "test_samples_per_second": 125.196,
  "test_steps_per_second": 1.996
}


Volvemos a clasificar las frases de antes y vemos que dan unos resultados mucho mejores con el entrenamiento.

In [7]:
# Cargamos el modelo ya con el FineTuning hecho
bert_class_model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=NUM_LABELS)
# Cargamos el Tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelo)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = bert_class_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= bert_class_model.config.id2label[predicted_class_id]
  print(text,'=>', predicted_class_id, '=>', prediction, "  ", logits.softmax(1))


hay muchos más muertos por covid => 0 => LABEL_0    tensor([[0.9953, 0.0047]])
el número de afectados por covid aumenta => 0 => LABEL_0    tensor([[0.9948, 0.0052]])
vamos a salir de la pandemia => 1 => LABEL_1    tensor([[0.0014, 0.9986]])
ánimo a todos => 1 => LABEL_1    tensor([[0.0010, 0.9990]])


## Apartado 1.3 Compatibilidad entre modelos Pytorch y Tensorflow

Se pueden cargar modelos de Pytorch en Tensorflow y viceversa. A continuación cargamos el modelo entrenado en Pytorch para evaluar las frases de los textos de antes y ver su clasificación.

In [8]:
import tensorflow as tf

# Probar a inferir nuevas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]


tf_model = TFAutoModelForSequenceClassification.from_pretrained('modeloReentrenadoPytorch')

# Imprimimos las predicciones obtenidas
for text in textos:
  predict_input = tokenizer.encode (text, truncation=True, padding=True, return_tensors="tf")
  tf_output = tf_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  print(text,'=>', tf_prediction)

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


1/1 [==============================] - 8s 8s/step
hay muchos más muertos por covid => [0.9953023  0.00469773]
1/1 [==============================] - 4s 4s/step
el número de afectados por covid aumenta => [0.9947536  0.00524635]
1/1 [==============================] - 0s 64ms/step
vamos a salir de la pandemia => [0.00135612 0.99864393]
1/1 [==============================] - 0s 67ms/step
ánimo a todos => [0.00104191 0.9989581 ]


## Apartado 1.4 Probamos a hacer Finetunning de un modelo "destilado" de BERT

En el siguiente bloque de código hacemos el mismo entrenamiento en Pytorch pero con el modelo 'CenIA/distillbert-base-spanish-uncased' que es más pequeño y rápido de entrenar


In [9]:
path_bert_model = 'CenIA/distillbert-base-spanish-uncased'

# Cargamos el modelo para clasificación en Pytorch
bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS)
# Cargamos el tokenizer de este modelo
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results',
  logging_dir = './logs',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size
)


tokenized_train_dataset = tokenizer (df_train.tweet.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.tweet.tolist (), truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.tweet.tolist (), truncation=True, padding = True)


# Como antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test._label.tolist())

from transformers import Trainer

trainer = Trainer (
    model = bert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()


print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Salvamos el modelo reentrenado
modelo ='modeloReentrenadoPytorch'
bert_class_model_pytorch.save_pretrained (modelo)
tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at CenIA/distillbert-base-spanish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.269100


PREDICCIONES SOBRE EVAL


{
  "eval_loss": 0.47665539383888245,
  "eval_accuracy": 0.8679245283018868,
  "eval_runtime": 3.3281,
  "eval_samples_per_second": 286.649,
  "eval_steps_per_second": 4.507,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.5137171745300293,
  "test_accuracy": 0.8531879194630873,
  "test_runtime": 5.5844,
  "test_samples_per_second": 213.451,
  "test_steps_per_second": 3.402
}


## Apartado 1.5 También se puede hacer el entrenamiento en **TensorFlow**

Os dejo en el siguiente bloque de código como se haría el entrenamiento en TensorFlow aunque es más recomendable y rápido hacerlo en Pytorch.

In [10]:
import tensorflow as tf

# Preparamos los conjuntos de entrenamiento, validación y test
train_encodings = tokenizer(df_train['tweet'].tolist (), truncation=True, padding=True, return_tensors='tf')
val_encodings = tokenizer(df_eval['tweet'].tolist (), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(df_test['tweet'].tolist (), truncation=True, padding=True, return_tensors='tf')


# Creamos los dataset de Tensorflow para poder hacer un finetuning del modelo de BERT
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), df_train['_label']))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), df_eval['_label']))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), df_test['_label']))

Definimos distintos parámetros de entrenamiento y entrenamos el modelo.

Es recomendable hacer este paso en casa porque tarda un tiempo considerable si no tienes una cuenta PRO de Google Colab

In [11]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

bert_class_model = TFAutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS, from_pt=True)
# Definimos en esta función algunos parámetros para el entrenamiento como los optimizadores y la función de pérdida
bert_class_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics='accuracy',
)

# Vamos a hacer el entrenamiento del modelo durante 3 épocas que es suficiente para que un modelo BERT de unos buenos resultados
EPOCHS = 3
BATCH_SIZE = 16
bert_class_model.fit(train_dataset.shuffle(100).batch(BATCH_SIZE),
          epochs=3,
          batch_size=BATCH_SIZE,
          validation_data=val_dataset.shuffle(100).batch(BATCH_SIZE),
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

Epoch 1/3
239/239 [==============================] - 132s 443ms/step - loss: 0.3599 - accuracy: 0.8392 - val_loss: 0.3339 - val_accuracy: 0.8574
Epoch 2/3
239/239 [==============================] - 108s 451ms/step - loss: 0.1815 - accuracy: 0.9326 - val_loss: 0.4115 - val_accuracy: 0.8690
Epoch 3/3
239/239 [==============================] - 107s 448ms/step - loss: 0.0875 - accuracy: 0.9719 - val_loss: 0.5504 - val_accuracy: 0.8522


Volvemos a clasificar las frases de antes y vemos que dan unos resultados mucho mejores con el entrenamiento.

In [12]:
# Probamos ahora de nuevo a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# Imprimimos las predicciones obtenidas
for text in textos:
  predict_input = tokenizer.encode (text, truncation=True, padding=True, return_tensors="tf")
  tf_output = bert_class_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  print(text,'=>', tf_prediction)


1/1 [==============================] - 2s 2s/step
hay muchos más muertos por covid => [0.9953681  0.00463194]
1/1 [==============================] - 1s 1s/step
el número de afectados por covid aumenta => [0.9373542  0.06264581]
1/1 [==============================] - 0s 30ms/step
vamos a salir de la pandemia => [0.0026415 0.9973585]
1/1 [==============================] - 0s 32ms/step
ánimo a todos => [0.00176343 0.99823654]


Es recomendable guardar los modelos reentrenados de BERT por si se quieren volver a utilizar o compartir.

In [13]:
# Podemos guardar el modelo con el Finetuning
bert_class_model.save_pretrained('modeloReentrenadoKeras')
# Para tener un modelo independiente deberíamos también guardar el Tokenizer aunque no lo hayamos modificado
tokenizer.save_pretrained('modeloReentrenadoKeras')

('modeloReentrenadoKeras/tokenizer_config.json',
 'modeloReentrenadoKeras/special_tokens_map.json',
 'modeloReentrenadoKeras/vocab.txt',
 'modeloReentrenadoKeras/added_tokens.json',
 'modeloReentrenadoKeras/tokenizer.json')

Validamos ahora con el conjunto de test y vemos que los resultados son los mejores que se han podido conseguir en cada uno de los experimentos realizados durante las sesiones.

In [14]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model = TFAutoModelForSequenceClassification.from_pretrained('modeloReentrenadoKeras')

# compile the model

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc',f1_m,precision_m, recall_m],
)
results=model.evaluate(test_dataset.batch(32))
print(results)

Some layers from the model checkpoint at modeloReentrenadoKeras were not used when initializing TFDistilBertForSequenceClassification: ['dropout_57']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at modeloReentrenadoKeras and are newly initialized: ['dropout_77']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


38/38 [==============================] - 12s 173ms/step - loss: 0.6268 - acc: 0.8490 - f1_m: 0.8333 - precision_m: 0.7280 - recall_m: 0.9830
[0.6267672181129456, 0.8489933013916016, 0.8332727551460266, 0.7279937863349915, 0.9829546213150024]
